In [1]:
import numpy as np
import numba as nb
from scipy.optimize import minimize
from scipy.integrate import quad

In [2]:
@nb.njit
def ch_func_1d(x, p, sb2, s02, n, r2_het_hist, nbin_r2_het_hist):
    fx = np.exp(-0.5*x*x*s02)
    for i in range(nbin_r2_het_hist):
        n_in_bin = r2_het_hist[i]
        if n_in_bin != 0:
            rh = (0.5*i + 0.25)/nbin_r2_het_hist # middle of the i-th hist bin with NBIN_R2_HET_HIST bins of [0, 0.5]
            se2 = n*sb2*rh
            fx *= (1 - p + p*np.exp(-0.5*x*x*se2))**n_in_bin
    return fx

@nb.njit
def ch_func_2d(x, y, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, pp, rho, rho0, n_1, n_2, r2_het_hist, nbin_r2_het_hist):
    fx = np.exp(-0.5*(s02_1*x*x + 2*rho0*np.sqrt(s02_1*s02_2)*x*y + s02_2*y*y))
    for i in range(nbin_r2_het_hist):
        n_in_bin = r2_het_hist[i]
        if n_in_bin != 0:
            rh = (0.5*i + 0.25)/nbin_r2_het_hist
            se2_1 = n_1*sb2_1*rh
            se2_2 = n_2*sb2_2*rh
            fx *= ( 1 - (p_1+p_2-pp) +
                    (p_1-pp)*np.exp(-0.5*x*x*se2_1) + 
                    (p_2-pp)*np.exp(-0.5*y*y*se2_2) +
                    pp*np.exp(-0.5*(se2_1*x*x + 2*rho*np.sqrt(se2_1*se2_2)*x*y + se2_2*y*y)) )**n_in_bin
    return fx

In [3]:
@nb.njit
def trapezoid(z0, s2):
    h = 5E-2
    a, fa = 0, 1
    tol = 1E-8
    res = 0
    while True:
        b = a + h
        ch_fb = ch_func1_test(b, s2)
        if ch_fb < tol:
            break
        else:
            fb = np.cos(z0*b)*ch_fb
            res += (fa + fb)
            a, fa = b, fb
    return 0.5*h*res

@nb.njit
def pdf_1d(z0, p, sb2, s02, n, r2_het_hist, nbin_r2_het_hist):
    # Characteristic function of sum of independent random variables = product of their characteristic functions.
    # Using inversion formula for characteristic function, pdf can be estimated as:
    # pdf = integral{0, inf}( cos(z0*t) * ch_func(t) )dt / pi
    # Integral is calculated using simple trapezoidal rule with fixed step size = h
    # Integration is stoped when ch_func(t) < tol
    h = 5E-2
    a, fa = 0, 1
    tol = 1E-8
    res = 0
    #TODO: implement similarly to 2d case as described here: https://math.stackexchange.com/questions/2891298/derivation-of-2d-trapezoid-rule
    while True:
        b = a + h
        ch_fb = ch_func_1d(b, p, sb2, s02, n, r2_het_hist, nbin_r2_het_hist)
        if ch_fb < tol:
            break
        else:
            fb = np.cos(z0*b)*ch_fb
            res += (fa + fb)
            a, fa = b, fb
    return 0.5*h*res / np.pi

@nb.njit(parallel=True)
def cost_1d(z0_vec, p, sb2, s02, n_vec, r2_het_hist, nbin_r2_het_hist):
    min_pdf = 1E-43
    cost = 0
    for i in nb.prange(z0_vec.shape[0]):
        z0, n = z0_vec[i], n_vec[i]
        pdf = pdf_1d(z0, p, sb2, s02, n, r2_het_hist, nbin_r2_het_hist)
        if pdf < min_pdf:
            pdf = min_pdf
        cost += -np.log(pdf)
    return cost

def obj_func_1d(par_vec, z0_vec, n_vec, r2_het_hist, nbin_r2_het_hist):
    p, sb2, s02 = par_vec
    p = 10**p
    sb2 = 10**sb2
    cost = cost_1d(z0_vec, p, sb2, s02, n_vec, r2_het_hist, nbin_r2_het_hist)
    print(f"cost = {cost:.7f}, p = {p:.3e}, sb2 = {sb2:.3e}, s02 = {s02:.4f}", flush=True)
    return cost

def optimize_1d(z0_vec, n_vec, r2_het_hist, nbin_r2_het_hist):
    p_lb, p_rb = -5, -2 # on log10 scale
    sb2_lb, sb2_rb = -6, -3 # on log10 scale
    s02_lb, s02_rb = 0.8, 2.5

    bounds = [(p_lb, p_rb), (sb2_lb, sb2_rb), (s02_lb, s02_rb)]
    x0 = [0.5*(b[0] + b[1]) for b in bounds]
    args_opt = (z0_vec, n_vec, r2_het_hist, nbin_r2_het_hist)
    res = minimize(obj_func_1d, x0=x0, args=args_opt, method='Nelder-Mead', bounds=bounds,
            options={'maxiter':200, 'fatol':1E-5, 'xatol':1E-2})
    
    opt_par = [10**res.x[0], 10**res.x[1], res.x[2]]
    opt_res = dict(x=res.x.tolist(), fun=res.fun.tolist())
    for k in ("success", "status", "message", "nfev", "nit"):
        opt_res[k] = res.get(k)
    opt_out = dict(opt_res=opt_res, opt_par=opt_par)
    return opt_out

@nb.njit
def pdf_2d(z0_1, z0_2, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, pp, rho, rho0, n_1, n_2, r2_het_hist, nbin_r2_het_hist):
    # integrate f(x,y) = cos(z0_1*x + z0_2*y)*cf(x, y) = cos(z0_1*x)*cos(z0_2*y)*cf(x, y) - sin(z0_1*x)sin(z0_2*y)*cf(x, y)
    # Note cf(x, y) = cf(-x, -y)
    # Take cos*cos*f and sin*sin*f integrals in [0, inf] x [0, inf] and in [0, inf] x [-inf, 0] quadrants simultaneously.
    # Integrals in [0, inf] x [0, inf] quadrant, are also equal to [-inf 0] x [-inf 0] quadrant.
    # Integrals in [0, inf] x [-inf, 0] quadrant, are also equal to [-inf 0] x [0 inf] quadrant.
    # Based on: https://math.stackexchange.com/questions/2891298/derivation-of-2d-trapezoid-rule
    h = 5E-2
    tol = 1E-8
    res = 1 # f(0,0) == 1
    # calculate sum[f(xi,0)]
    x, y = 0, 0
    while True:
        x = x + h
        ch_f = ch_func_2d(x, y, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, pp, rho, rho0, n_1, n_2, r2_het_hist, nbin_r2_het_hist)
        if ch_f < tol:
            break
        else:
            fx = np.cos(z0_1*x + z0_2*y)*ch_f
            res += 2*fx
    # calculate sum[f(0,yi)]
    x, y = 0, 0
    while True:
        y = y + h
        ch_f = ch_func_2d(x, y, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, pp, rho, rho0, n_1, n_2, r2_het_hist, nbin_r2_het_hist)
        if ch_f < tol:
            break
        else:
            fx = np.cos(z0_1*x + z0_2*y)*ch_f
            res += 2*fx      
    # calculate sum[f(xi,yi)]
    x, y = 0, 0
    while True:
        y = y + h
        n_x_steps = 0
        while True:
            x = x + h
            ch_f = ch_func_2d(x, y, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, pp, rho, rho0, n_1, n_2, r2_het_hist, nbin_r2_het_hist)
            if ch_f < tol:
                break
            else:
                fx = np.cos(z0_1*x + z0_2*y)*ch_f
                res += 4*fx
                n_x_steps += 1
        if n_x_steps == 0:
            break
    # Now res = integral{0,inf; 0,inf} == integral{-inf,0; -inf,0}
    # Estimate integral{0,inf; -inf, 0}
    # calculate sum[f(0,-yi)]
    x, y = 0, 0
    while True:
        y = y - h
        ch_f = ch_func_2d(x, y, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, pp, rho, rho0, n_1, n_2, r2_het_hist, nbin_r2_het_hist)
        if ch_f < tol:
            break
        else:
            fx = np.cos(z0_1*x + z0_2*y)*ch_f
            res += 2*fx      
    # calculate sum[f(-xi,-yi)]
    x, y = 0, 0
    while True:
        y = y - h
        n_x_steps = 0
        while True:
            x = x - h
            ch_f = ch_func_2d(x, y, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, pp, rho, rho0, n_1, n_2, r2_het_hist, nbin_r2_het_hist)
            if ch_f < tol:
                break
            else:
                fx = np.cos(z0_1*x + z0_2*y)*ch_f
                res += 4*fx
                n_x_steps += 1
        if n_x_steps == 0:
            break
    # Now res = integral{0,inf; 0,inf} + integral{0,inf; -inf,0}
    # final value should be = h^2/4 * (2*res/(2*pi)**2) = h*h*res / np.pi**2
    return h*h*res / np.pi**2

@nb.njit(parallel=True)
def cost_2d(z0_1_vec, z0_2_vec, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, pp, rho, rho0, n_1_vec, n_2_vec,
            r2_het_hist, nbin_r2_het_hist):
    min_pdf = 1E-43
    cost = 0
    for i in nb.prange(z0_1_vec.shape[0]):
        z0_1, n_1, z0_2, n_2 = z0_1_vec[i], n_1_vec[i], z0_2_vec[i], n_2_vec[i]
        pdf = pdf_2d(z0_1, z0_2, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, pp, rho, rho0, n_1, n_2, r2_het_hist, nbin_r2_het_hist)
        if pdf < min_pdf:
            pdf = min_pdf
        cost += -np.log(pdf)
    return cost

def obj_func_2d(par_vec, z0_1_vec, z0_2_vec, n_1_vec, n_2_vec, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, r2_het_hist, nbin_r2_het_hist):
    pp, rho, rho0 = par_vec
    pp = 10**pp
    
    cost = cost_2d(z0_1_vec, z0_2_vec, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, pp, rho, rho0, n_1_vec, n_2_vec,
            r2_het_hist, nbin_r2_het_hist)
    print(f"cost = {cost:.7f}, p12 = {pp:.3e}, rho = {rho:.4f}, rho0 = {rho0:.4f}", flush=True)
    return cost

def optimize_2d(p_1, sb2_1, s02_1, n_1_vec, z0_1_vec, p_2, sb2_2, s02_2, n_2_vec,
                z0_2_vec, r2_het_hist, nbin_r2_het_hist):
    p12_lb, p12_rb = -6, np.log10(min(p_1,p_2)) # on log10 scale
    assert p12_lb < p12_rb
    rho_lb, rho_rb = -1, 1
    rho0_lb, rho0_rb = -1, 1
     
    bounds = [(p12_lb, p12_rb), (rho_lb, rho_rb), (rho0_lb, rho0_rb)]
    x0 = [0.5*(b[0] + b[1]) for b in bounds]
    args_opt = (z0_1_vec, z0_2_vec, n_1_vec, n_2_vec, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, r2_het_hist, nbin_r2_het_hist)
    res = minimize(obj_func_2d, x0=x0, args=args_opt, method='Nelder-Mead', bounds=bounds,
            options={'maxiter':300, 'fatol':1E-5, 'xatol':1E-2})
    
    opt_par = [10**res.x[0], res.x[1], res.x[2]]
    opt_res = dict(x=res.x.tolist(), fun=res.fun.tolist())
    for k in ("success", "status", "message", "nfev", "nit"):
        opt_res[k] = res.get(k)
    opt_out = dict(opt_res=opt_res, opt_par=opt_par)
    return opt_out

In [5]:
import mix3r_v2

In [6]:
config_fname = "/home/alexeas/Documents/github/mix3r/data/config/scz_th_ms_config_1.json"
args = mix3r_v2.parse_args(f"--config {config_fname}".split())
config =  mix3r_v2.load_config(args)

In [7]:
snps_df = mix3r_v2.load_snps(config["template_dir"], config["sumstats"],
        chromosomes=config["snp_filters"]["chromosomes"],
        z_thresh=config["snp_filters"]["z_thresh"],
        info_thresh=config["snp_filters"]["info_thresh"],
        maf_thresh=config["snp_filters"]["maf_thresh"],
        exclude_regions=config["snp_filters"]["exclude_regions"])

Reading template SNPs for 2 chromosomes from /home/alexeas/Documents/github/mix3r/data/template/ukb
    342573 SNPs
Loading sumstats from /home/alexeas/Documents/github/mix3r/data/sumstats/PGC_SCZ_0518_EUR.sumstats.gz
    7617200 SNPs
Loading sumstats from /home/alexeas/Documents/github/mix3r/data/sumstats/ENIGMA_TH_2020_noGC.sumstats.gz
    8599494 SNPs
Loading sumstats from /home/alexeas/Documents/github/mix3r/data/sumstats/IMSGC_MS_2019.sumstats.gz
    8765578 SNPs
182964 common SNPs
182964 SNPs with matched alleles
182964 SNPs with Z < 32
160850 SNPs with INFO > 0.9
129829 SNPs with MAF > 0.05
    0 SNPs excluded from 6:25000000-33000000
    0 SNPs excluded from 8:7200000-12500000
    0 SNPs excluded from 17:40000000-47000000
    0 SNPs excluded from 19:42000000-47000000
129829 SNPs after all filters


In [8]:
r2_het_hist, z_n_dict =  mix3r_v2.load_opt_data(config["template_dir"], snps_df,
        r2_prune_thresh=config["pruning"]["r2_prune_thresh"],
        rand_prune_seed=config["pruning"]["rand_prune_seed"])

Loading LD data
Processing chr 21
    3544 SNPs survive pruning
    722.37 mean size of LD block of pruned SNPs
Processing chr 22
    3657 SNPs survive pruning
    697.68 mean size of LD block of pruned SNPs
7201 SNPs loaded
709.84 mean size of LD block of loaded SNPs


In [9]:
mix3r_v2.NBIN_R2_HET_HIST, r2_het_hist.shape[0]/mix3r_v2.NBIN_R2_HET_HIST

(64, 7201.0)

In [17]:
opt_out = optimize_1d(z_n_dict["Z_2"], z_n_dict["N_2"], r2_het_hist, mix3r_v2.NBIN_R2_HET_HIST)

cost = 11214.8415845, p = 3.162e-04, sb2 = 3.162e-05, s02 = 1.6500
cost = 11209.0836042, p = 2.113e-04, sb2 = 3.162e-05, s02 = 1.6500
cost = 11208.3654245, p = 3.162e-04, sb2 = 1.884e-05, s02 = 1.6500
cost = 11260.2038845, p = 3.162e-04, sb2 = 3.162e-05, s02 = 1.7325
cost = 11167.3920416, p = 2.417e-04, sb2 = 2.239e-05, s02 = 1.5675
cost = 11131.8114900, p = 2.113e-04, sb2 = 1.884e-05, s02 = 1.4850
cost = 11152.0212807, p = 1.848e-04, sb2 = 1.585e-05, s02 = 1.5400
cost = 11123.9266532, p = 2.528e-04, sb2 = 1.000e-05, s02 = 1.4667
cost = 11098.0489502, p = 2.765e-04, sb2 = 5.623e-06, s02 = 1.3750
cost = 11086.6584128, p = 1.545e-04, sb2 = 7.499e-06, s02 = 1.2833
cost = 11125.9321676, p = 1.080e-04, sb2 = 4.732e-06, s02 = 1.1000
cost = 11088.8876030, p = 2.346e-04, sb2 = 5.412e-06, s02 = 1.2222
cost = 11125.0803632, p = 2.199e-04, sb2 = 1.983e-06, s02 = 1.1020
cost = 11092.7633546, p = 2.178e-04, sb2 = 3.481e-06, s02 = 1.1978
cost = 11128.7503449, p = 1.434e-04, sb2 = 4.823e-06, s02 = 1.

cost = 11066.0628263, p = 1.984e-04, sb2 = 1.672e-04, s02 = 1.1492
cost = 11066.0013761, p = 1.807e-04, sb2 = 1.611e-04, s02 = 1.1631
cost = 11065.9096430, p = 1.727e-04, sb2 = 1.686e-04, s02 = 1.1606
cost = 11065.8627156, p = 1.635e-04, sb2 = 1.736e-04, s02 = 1.1614
cost = 11065.8984275, p = 1.723e-04, sb2 = 1.750e-04, s02 = 1.1554
cost = 11066.0116073, p = 1.581e-04, sb2 = 1.794e-04, s02 = 1.1518
cost = 11065.9280020, p = 1.791e-04, sb2 = 1.678e-04, s02 = 1.1596
cost = 11065.8161860, p = 1.635e-04, sb2 = 1.782e-04, s02 = 1.1634
cost = 11065.7670165, p = 1.558e-04, sb2 = 1.846e-04, s02 = 1.1679
cost = 11065.7401572, p = 1.497e-04, sb2 = 1.881e-04, s02 = 1.1636
cost = 11065.6730117, p = 1.369e-04, sb2 = 1.992e-04, s02 = 1.1656
cost = 11065.6408454, p = 1.334e-04, sb2 = 1.966e-04, s02 = 1.1745
cost = 11065.5924998, p = 1.174e-04, sb2 = 2.083e-04, s02 = 1.1840
cost = 11065.5210702, p = 1.128e-04, sb2 = 2.238e-04, s02 = 1.1836
cost = 11065.5127652, p = 9.372e-05, sb2 = 2.541e-04, s02 = 1.

In [18]:
opt_out

{'opt_res': {'x': [-4.115098368070832, -3.5486098779188158, 1.192685791847849],
  'fun': 11065.383098723925,
  'success': True,
  'status': 0,
  'message': 'Optimization terminated successfully.',
  'nfev': 191,
  'nit': 106},
 'opt_par': [7.671877010118643e-05, 0.0002827418674411276, 1.192685791847849]}

In [28]:
x = 1
y = 0.1
p_1 = 0.003
p_2 = 0.001
sb2_1 = 2E-5
sb2_2 = 1E-4
s02_1 = 0.9
s02_2 = 1.4
pp = 0.0005
rho = -0.2
rho0 = 0.02
n_1 = 100000
n_2 = 25000
nbin_r2_het_hist = mix3r_v2.NBIN_R2_HET_HIST
ch_func_2d(x, y, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, pp, rho, rho0, n_1, n_2, r2_het_hist, nbin_r2_het_hist)

0.49142373382929244

In [36]:
z0_1 = -0.1
z0_2 = 2.3
pdf_2d(z0_1, z0_2, p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, pp, rho, rho0, n_1, n_2, r2_het_hist, nbin_r2_het_hist)

0.05698365728794713

In [55]:
cost_2d(z_n_dict["Z_0"], z_n_dict["Z_1"], p_1, p_2, sb2_1, sb2_2, s02_1, s02_2, pp, rho, rho0,
        z_n_dict["N_0"], z_n_dict["N_1"], r2_het_hist, nbin_r2_het_hist)

22451.36325762541

In [64]:
optimize_2d(p_1, sb2_1, s02_1, z_n_dict["N_0"], z_n_dict["Z_0"], p_2, sb2_2, s02_2, z_n_dict["N_1"], z_n_dict["Z_1"], r2_het_hist, nbin_r2_het_hist)

cost = 22451.5804523, p12 = 3.162e-05, rho = 0.0000, rho0 = 0.0000
cost = 22451.5816382, p12 = 1.884e-05, rho = 0.0000, rho0 = 0.0000
cost = 22451.5805835, p12 = 3.162e-05, rho = 0.0003, rho0 = 0.0000
cost = 22451.5990676, p12 = 3.162e-05, rho = 0.0000, rho0 = 0.0003
cost = 22451.5627556, p12 = 2.239e-05, rho = 0.0002, rho0 = -0.0003
cost = 22451.5444860, p12 = 1.884e-05, rho = 0.0003, rho0 = -0.0005
cost = 22451.5552871, p12 = 3.758e-05, rho = 0.0003, rho0 = -0.0003
cost = 22451.5397464, p12 = 2.512e-05, rho = 0.0001, rho0 = -0.0006
cost = 22451.5192893, p12 = 2.239e-05, rho = 0.0001, rho0 = -0.0008
cost = 22451.4989480, p12 = 1.995e-05, rho = 0.0004, rho0 = -0.0011
cost = 22451.4579902, p12 = 1.585e-05, rho = 0.0007, rho0 = -0.0017
cost = 22451.4584614, p12 = 9.441e-06, rho = 0.0003, rho0 = -0.0017
cost = 22451.4127727, p12 = 1.189e-05, rho = 0.0005, rho0 = -0.0023
cost = 22451.3468121, p12 = 9.441e-06, rho = 0.0006, rho0 = -0.0032
cost = 22451.3223460, p12 = 5.623e-06, rho = 0.0010,

{'opt_res': {'x': [-6.0, -1.0, -1.0],
  'fun': 22377.635581982464,
  'success': True,
  'status': 0,
  'message': 'Optimization terminated successfully.',
  'nfev': 62,
  'nit': 34},
 'opt_par': [1e-06, -1.0, -1.0]}

In [4]:
print(nb.__version__)

0.53.1


In [5]:
trapezoid(0.12, 1.2)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
NameError: name 'ch_func1_test' is not defined